### Step 1: Mount the Google Drive

Remember to use GPU runtime before mounting your Google Drive. (Runtime --> Change runtime type).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Step 2: Open the project directory

Replace `Your_Dir` with your own path.

In [ ]:
cd Your_Dir/emg2qwerty

### Step 3: Install required packages

After installing them, Colab will require you to restart the session.

In [ ]:
!pip install -r requirements.txt

### Step 4: Start your experiments!

- Remember to download and copy the dataset to this directory: `Your_Dir/emg2qwerty/data`.
- You may now start your experiments with any scripts! Below are examples of single-user training and testing (greedy decoding).
- **There are two ways to track the logs:**
  - 1. Keep `--multirun`, and the logs will not be printed here, but they will be saved in the folder `logs`, e.g., `logs/2025-02-09/18-24-15/submitit_logs/`.
  - 2. Comment out `--multirun` and the logs will be printed in this notebook, but they will not be saved.

#### Training

- The checkpoints are saved in the folder `logs`, e.g., `logs/2025-02-09/18-24-15/checkpoints/`.

In [ ]:
# Please note the following:
#  - I've added PYTHONPATH because my environment has some bugs --> please remove it before running
#  - I run with batch_size=8 to avoid CUDA OOM failures
#  - CUDA still throws an OOM at test-time, even if training succeeded

# Single-user training
!PYTORCH_ENABLE_MPS_FALLBACK=1 \
  HYDRA_FULL_ERROR=1 \
  PYTHONPATH=/opt/conda/lib/python3.10/site-packages \
  python -m emg2qwerty.train \
  user="single_user" \
  trainer.accelerator=gpu \
  trainer.devices=1 \
  trainer.max_epochs=40 \
  batch_size=8 \
  --multirun

[2025-03-08 02:22:21,223][HYDRA] Submitit 'local' sweep output dir : logs/2025-03-08/02-22-20
[2025-03-08 02:22:21,226][HYDRA] 	#0 : user=single_user trainer.accelerator=gpu trainer.devices=1 trainer.max_epochs=40 batch_size=8
Error executing job with overrides: ['user=single_user', 'trainer.accelerator=gpu', 'trainer.devices=1', 'trainer.max_epochs=40', 'batch_size=8']
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/workspace/emg2qwerty/train.py", line 129, in <module>
    main()
  File "/home/user/.local/lib/python3.10/site-packages/hydra/main.py", line 94, in decorated_main
    _run_hydra(
  File "/home/user/.local/lib/python3.10/site-packages/hydra/_internal/utils.py", line 394, in _run_hydra
    _run_app(
  File "/home/user/.local/lib/python3.10/site-packages/hydra/_in

#### Testing:

- Replace `Your_Path_to_Checkpoint` with your checkpoint path.

In [15]:
# Single-user testing
!PYTORCH_ENABLE_MPS_FALLBACK=1 \
  HYDRA_FULL_ERROR=1 \
  CUDA_LAUNCH_BLOCKING=1 \
  python -m emg2qwerty.train \
  user="single_user" \
  checkpoint="'/workspace/logs/2025-03-08/02-22-20/job0_batch_size=8,trainer.devices=1,trainer.max_epochs=40,user=single_user/checkpoints/epoch=37-step=18240.ckpt'" \
  train=False trainer.accelerator=cpu \
  decoder=ctc_greedy \
  hydra.launcher.mem_gb=64 \
  # --multirun

[2025-03-08 05:26:42,410][__main__][INFO] - 
Config:
user: single_user
dataset:
  train:
  - user: 89335547
    session: 2021-06-03-1622765527-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f
  - user: 89335547
    session: 2021-06-02-1622681518-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f
  - user: 89335547
    session: 2021-06-04-1622863166-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f
  - user: 89335547
    session: 2021-07-22-1627003020-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f
  - user: 89335547
    session: 2021-07-21-1626916256-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f
  - user: 89335547
    session: 2021-07-22-1627004019-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f
  - user: 89335547
    session: 2021-06-05-1622885888-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f
  - user: 89335547
    session: 2021-06-02-1622679967-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f

In [ ]:
!tensorboard --logdir='/Users/jonathonyoungquist/Local/ECE_C147/Project/emg2qwerty/logs/2025-03-01/11-14-51/job0_trainer.devices=1,trainer.max_epochs=40,user=single_user/lightning_logs/version_0/'